In [ ]:
import os, sys, time

import requests
import shutil

# https://gitlab.com/dlr-ve-esy/esy-osmfilter/-/tree/master/
from esy.osmfilter import  osm_colors          as CC
from esy.osmfilter import run_filter, Node,Way,Relation 
from esy.osmfilter import export_geojson
from contextlib import contextmanager

import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import geoplot
import matplotlib.pyplot as plt
# import logging
# logging.basicConfig()
# logger=logging.getLogger(__name__)
# logger.setLevel(logging.INFO)
# logger.setLevel(logging.WARNING)


In [ ]:
#geofabrik_regions = ["algeria","angola","benin","botswana","burkina_faso","burundi","cameroon","canary_islands","cape_verde","central_african_republic","chad","comores","congo_brazzaville","congo_democratic_republic","djibouti","egypt","equatorial_guinea","eritrea","ethiopia","gabon","ghana","guinea_bissau","guinea","ivory_coast","kenya","lesotho","liberia","libya","madagascar","malawi","mali","mauritania","mauritius","morocco","mozambique","namibia","niger","nigeria","rwanda","saint_helena_ascension_and_tristan_da_cunha","sao_tome_and_principe","senegal_and_gambia","seychelles","sierra_leone","somalia","south_africa_and_lesotho","south_africa","south_sudan","sudan","swaziland","tanzania","togo","tunisia","uganda","zambia","zimbabwe"]

In [ ]:
country_name = 'egypt' # TODO: Create dictionary with country_code : country_name pairs
country_code = 'egypt'
update = False # prevents re-download and filteration of file
element_file_exists = False

# json file for the Data dictionary 
# json file for the Elements dictionary is automatically written to 'data/osm/Elements'+filename)
JSON_outputfile = os.path.join(os.getcwd(),'data','osm',country_name+'_generator.json')

if os.path.exists(JSON_outputfile):
    element_file_exists = True

geofabrik_filename=f'{country_name}-latest.osm.pbf' # Filename for geofabrik 
geofabrik_url = f'https://download.geofabrik.de/africa/{geofabrik_filename}' # https://download.geofabrik.de/africa/nigeria-latest.osm.pbf

PBF_inputfile = os.path.join(os.getcwd(),"data","osm","pbf",geofabrik_filename) # Input filepath

if not os.path.exists(PBF_inputfile) or update or element_file_exists is False:
    print(f"{geofabrik_filename} does not exist, downloading to {PBF_inputfile}")
    os.makedirs(os.path.dirname(PBF_inputfile), exist_ok=True) # create data/osm directory
    with requests.get(geofabrik_url, stream=True) as r:
        with open(PBF_inputfile, 'wb') as f:
            shutil.copyfileobj(r.raw, f)





In [ ]:
print(PBF_inputfile, JSON_outputfile)

In [ ]:
#see https://dlr-ve-esy.gitlab.io/esy-osmfilter/filter.html for filter structures

prefilter = {Node: {"power":["generator"]}, Way: {"power":["generator"]}, Relation: {"power":["generator"]}}
blackfilter = [("pipeline","generator"),] # HACKY: due to esy.osmfilter
whitefilter =[[("power","generator"),],]
    
elementname=f'{country_code}_substations'

if update is False or element_file_exists is False:
    create_elements = False # Do not create elements again
    new_prefilter_data = False #Do not pre-filter data again
    print("Loading Pickle")
else :
    create_elements = True # Do not create elements again
    new_prefilter_data = True #Do not pre-filter data again
    print("Creating  New Elements")

[Data,Elements]=run_filter(elementname,PBF_inputfile, JSON_outputfile,prefilter,whitefilter,blackfilter, NewPreFilterData=new_prefilter_data, CreateElements=create_elements, LoadElements=True,verbose=False,multiprocess=True)

# export the elements to a geojson-file
# export_geojson(Elements[elementname]['Way'],Data,filename='test.geojson',jsontype='Line')
# convert extract node elements
# export_geojson(Elements[elementname]['Node'],Data,filename='test.geojson',jsontype='Point')

In [ ]:
df_way = pd.json_normalize(Elements[elementname]["Node"].values())
#df_way.dropna(subset=['tags.voltage'], inplace = True) # Drop any substations with Voltage = N/A
# na_values_voltage = df_way.isna().sum()
#df_way.dropna(thresh=len(df_way)*0.25, axis=1, how='all', inplace = True) #Drop Columns with 75% values as N/A
df_way.drop(columns = "tags.fixme", inplace = True)
df_way.drop(columns = "tags.frequency", inplace = True)
display(df_way)
df_way.lonlat

In [ ]:
df_way = pd.json_normalize(Elements[elementname]["Way"].values())
#df_way.dropna(subset=['tags.voltage'], inplace = True) # Drop any substations with Voltage = N/A
# na_values_voltage = df_way.isna().sum()
#df_way.dropna(thresh=len(df_way)*0.25, axis=1, how='all', inplace = True) #Drop Columns with 75% values as N/A
df_way.drop(columns = "tags.frequency", inplace = True)
df_way.drop(columns = "tags.barrier", inplace = True)
df_way.drop(columns = "tags.building", inplace = True)
df_way.drop(columns = "tags.name:ar", inplace = True)
#display(df_way)

In [ ]:
# # Convert Ways to Nodes
# lonlat_column =[]
# for ref in df_way["refs"]:
#     lonlats =[]
#     for r in ref:
#         lonlat = Data["Node"][str(r)]["lonlat"]
#         lonlats.append(lonlat)
#     lonlats = np.array(lonlats)
#     lonlat = np.mean(lonlats, axis=0) #Hacky Apporx Centroid
#     lonlat_column.append(lonlat)
# df_way.drop('refs',axis=1,inplace=True)
# df_way.insert(1, "lonlat", lonlat_column)
# display(df_way)
    
# # polygon_column=[]
# # for poly_list in lonlat_column:
# #     for p in poly_list:
# #         print(p)

# #     print("p")
    

In [ ]:
# pd.merge(df_way, df_way1, on='id')#,'tags.generator:type','tags.power','tags.generator:method','tags.generator:source'])#,  'tags.generator:source','tags.generator:output:electricity', 'tags.name'])#, on=["STATION", "DATE"])

# #df_way.columns[:8]



In [ ]:
gdf = gpd.GeoDataFrame(df_way, geometry=[Point(x, y) for x, y in df_way.lonlat])
gdf.drop(columns=['lonlat'], inplace=True)
display(gdf)

In [ ]:
GeoJSON_outputfile = os.path.join(os.getcwd(),'data','osm',country_name+'_generators.geojson')
gdf.to_file(GeoJSON_outputfile, driver="GeoJSON")  

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
saudi = world.query('name == "Saudi Arabia"')
yemen = world.query('name == "Yemen"')
iraq = world.query('name == "Iraq"')
oman = world.query('name == "Oman"')
uae = world.query('name == "United Arab Emirates"')
jordan = world.query('name == "Jordan"')
palestine = world.query('name == "Palestine"')
israel = world.query('name == "Israel"')
qatar = world.query('name == "Qatar"')
kuwait = world.query('name == "Kuwait"')
mena = saudi.append(yemen).append(iraq).append(oman).append(uae).append(jordan).append(palestine).append(israel).append(qatar).append(kuwait)
africa = world.query('continent == "Africa"').append(mena)

In [ ]:
url = 'https://development-data-hub-s3-public.s3.amazonaws.com/ddhfiles/144823/africagrid20170906final.geojson'
africa_network = gpd.read_file(url)
africa_generators = gpd.read_file(GeoJSON_outputfile)

In [ ]:
ax = africa_network[africa_network.voltage_kV > 110].plot(figsize=(20, 20),alpha = 0.7)
geoplot.polyplot(africa ,edgecolor='grey', ax=ax, alpha = 0.5)
africa_substations.plot(figsize=(20, 20),markersize=25, color='orange',alpha = 0.5, ax=ax)
#plt.savefig('africa_transmission_and substations_110.png')